# Bayesian Cosmology: Measuring H₀ with MCMC

AST 3414 - Spring 2026

## Introduction

In this lab, you'll use **Bayesian inference** and **Markov Chain Monte Carlo (MCMC)** to measure the Hubble constant from supernova data which we previously fit with $\chi^2$.

### What You'll Learn

1. **Bayesian inference** - Sample the full posterior distribution, and allows for flexibility in constructing a generative model for your data
2. **Prior selection** - Choose and justify reasonable priors
3. **Posterior analysis** - Interpret MCMC chains and corner plots

---

## Bayes' Theorem

$$P(\theta | D) = \frac{P(D | \theta) \cdot P(\theta)}{P(D)}$$

**Translation**:
$$\text{Posterior} = \frac{\text{Likelihood} \times \text{Prior}}{\text{Evidence}}$$

For parameter estimation, we can ignore the evidence (normalization):

$$P(\theta | D) \propto P(D | \theta) \cdot P(\theta)$$

**What we need to define**:
1. **Likelihood** $P(D | \theta)$: How probable is our data given parameters?
2. **Prior** $P(\theta)$: What do we know about parameters before seeing data?

MCMC then samples from the posterior to give us the full probability distribution of our parameters!

---

# Notes about MCMC

Previously, you explores a Bayesian solution to fit a straight line using a mixture model. As you add parameters to a model you encounter the "curse of dimensionality." Consider how many model evaluations you woulld need to for an N-dimensional gride with 100 samples per dimension.

In one dimension (one parameter), you have 100 points.

In two dimensions, you have $100^2 = 10,000$ evaluations.

In three dimensions, you have $100^3 = 1,000,000$ evaluations.

In $N$ dimensions, you have $100^N$ evaluations, and as $N$ grows this quickly becomes untenable with a grid search. The idea that made Bayesian modeling possible and practicle is *Markov Chain Monte Carlo (MCMC)*. This approach efficiently draws random samples from a posterior distribution even in relatively high dimensions.

Look at the simplest MCMC sampling algorithm, the *Metropolis-Hastings Sampler*. This procedure draws psuedo-random chains of points which, in the long-term limit, are a representative sample from the posterior. The procedure is very simple:

1. Define a posterior $p(\theta|D,I)$
2. Define a proposal density $p(\theta_{i+1}|\theta_i)$, which must be a symmetric function, but otherwise is unconstrained (usually a Gaussian is chosen).
3. Choose a starting point $\theta_0$
4. Repeat the following:
   1. Given $\theta_i$, draw a new $\theta_{i+1}$
   2. Compute the acceptance ratio, $a = \frac{p(\theta_{i+1}|D,I)}{p(\theta_{i}|D,I)}$.
   3. If $a\geq1$, the proposal is more likely: accept the draw and add $\theta_{i+1}$ to the chain.
   4. If $a\lt 1$, then accept the point with probabilbity $a$. This can be done by drawing a uniform random number $r$ and checking if $a
   \lt r$. If the point is accepted, add $\theta_{i+1}$ to the chain. If not, then add $\theta_i$ to the chain *again*.

This works very well! Multiple "walkers" can explore parameter space in parallel, which is efficient for moderate-dimensional problems (~10-20 parameters). Rather than climbing to a peak (optimization), the idea is to map out the entire landscape (sampling).

Modern MCMC algorithms, such as **emcee** (Foreman-Mackey et al. 2013) which we are using, have some improvements to ensure efficient scaling of the posterior, but are not much different.  But there are a few caveats you should be aware of.

##1. The procedure is only correct in the long-term limit
Somethimes the long-term limit is **very** long. You want to find "stabilization" of the MCMC chain, meaning that it has reached a statistical equilibrium. Here we will just look at the chains to make sure stabilization has been reached.

##2. The size of the proposal distribution is **very** important
If your proposal distribution is too small, it will take too long for your chain to move, and you have the danger of getting stuck in a local maximum. If your proposal distribution is too large, you will not be able to efficiently explore the sapace around a particular peak.

In general, choosing an appropriate scale for the proposal distribution is one of the most difficult parts of using the MCMC procedure. More sophisticated methods (such as **emcee**) have built-in ways to estimate the appropriate size of the proposal distribution.

##3. Fast stabilization can be helped by a good initialization
To ensure that the MCMC stabilizes quickly, start with a value you know is relatively close to the maximum a posteriori value, or the peak in the posterior distribution.


## Part 1: Setup and Data Loading

We'll need:
- `numpy`, `matplotlib`: Standard scientific Python
- `scipy`: For numerical integration  
- `emcee`: MCMC sampler
- `corner`: Visualization of posterior distributions
- `pandas`: Data loading

In [ ]:
# Install packages (Colab doesn't have emcee or corner by default)
!pip install -q emcee corner

In [ ]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import quad
from scipy.stats import norm
import emcee
import corner
import pandas as pd

# Set random seed for reproducibility
np.random.seed(42)

# Plot settings
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

print("✓ All packages loaded successfully!")
print(f"✓ emcee version: {emcee.__version__}")

### Load Supernova Data

We'll use the Pantheon+SH0ES dataset which you can download from my github using the code below. As a reminder, the Pantheon+SH0ES contains ~1700 Type Ia supernovae with: redshift (z) and uncertainty (σ_z), and distance modulus (μ) and uncertainty (σ_μ).


In [ ]:
!wget https://raw.githubusercontent.com/profhewitt/ast3414/refs/heads/main/Pantheon%2BSH0ES.dat

In [ ]:
data = pd.read_csv('Pantheon+SH0ES.dat', sep='\\s+', comment='#')

# Extract columns (adjust names as needed based on actual file)
if 'zHD' in data.columns:
    z_obs = data['zHD'].values
    mu_obs = data['MU_SH0ES'].values
    sigma_mu = data['MU_SH0ES_ERR_DIAG'].values
    sigma_z = 0.001 * np.ones_like(z_obs)  # Typical uncertainty
else:
    # Generic column access
    z_obs = data.iloc[:, 1].values
    mu_obs = data.iloc[:, 4].values
    sigma_mu = data.iloc[:, 5].values
    sigma_z = 0.001 * np.ones_like(z_obs)

print(f"✓ Loaded {len(z_obs)} real supernovae from Pantheon+SH0ES")
data_source = "real"

# Summary
print(f"\nData summary:")
print(f"  Redshift range: {z_obs.min():.4f} to {z_obs.max():.4f}")
print(f"  Distance modulus range: {mu_obs.min():.2f} to {mu_obs.max():.2f}")
print(f"  Mean σ_z: {sigma_z.mean():.5f}")
print(f"  Mean σ_μ: {sigma_mu.mean():.3f}")

### Visualize the Data

Let's look at our Hubble diagram and understand the data.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Left: Hubble diagram
ax1 = axes[0]
ax1.errorbar(z_obs, mu_obs, yerr=sigma_mu, xerr=sigma_z,
             fmt='o', alpha=0.4, markersize=3, elinewidth=0.5,
             label=f'{len(z_obs)} supernovae')
ax1.set_xlabel('Redshift (z)', fontsize=14)
ax1.set_ylabel('Distance Modulus (μ)', fontsize=14)
ax1.set_title('Hubble Diagram', fontsize=16)
ax1.legend()
ax1.grid(True, alpha=0.3)

# Right: Uncertainty distributions
ax2 = axes[1]
ax2.hist(sigma_z * 1000, bins=30, alpha=0.6, label='σ_z (×10⁻³)', color='blue')
ax2.hist(sigma_mu, bins=30, alpha=0.6, label='σ_μ', color='red')
ax2.set_xlabel('Uncertainty', fontsize=14)
ax2.set_ylabel('Count', fontsize=14)
ax2.set_title('Error Distributions', fontsize=16)
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---
## Part 2: The Model

We'll fit a flat ΛCDM cosmology with two parameters:
- $H_0$: Hubble constant (km/s/Mpc)
- $\Omega_m$: Matter density parameter

With the constraint: $\Omega_\Lambda = 1 -\Omega_m$ (flat universe)

### Luminosity Distance

For a flat universe, the luminosity distance is:

$$d_L(z) = \frac{c(1+z)}{H_0} \int_0^z \frac{dz'}{E(z')}$$

where:

$$E(z) = \sqrt{\Omega_m(1+z)^3 + \Omega_\Lambda}$$

and c = 299,792.458 km/s.

### Distance Modulus

$$\mu(z) = 5\log_{10}\left(\frac{d_L(z)}{10\,\text{pc}}\right) = 5\log_{10}(d_L) + 25$$

where $d_L$ is in Mpc.

Let's implement this model.

In [ ]:
# Speed of light in km/s
c_light = 299792.458

def E(z, Omega_m):
    Omega_Lambda = 1 - Omega_m
    return np.sqrt(Omega_m * (1 + z)**3 + Omega_Lambda)

def luminosity_distance(z, H0, Omega_m):
    z_array = np.atleast_1d(z)
    d_L = np.zeros_like(z_array, dtype=float)

    for i, zi in enumerate(z_array):
        # Integrate 1/E(z) from 0 to zi
        integrand = lambda zp: 1.0 / E(zp, Omega_m)
        integral, _ = quad(integrand, 0, zi)

        # Calculate luminosity distance
        d_L[i] = (c_light / H0) * (1 + zi) * integral

    return d_L if z.shape else float(d_L[0])

def distance_modulus(z, H0, Omega_m):
    d_L = luminosity_distance(z, H0, Omega_m)
    return 5 * np.log10(d_L) + 25

# Test the model
test_z = np.array([0.1, 0.5, 1.0])
test_H0 = 70
test_Om = 0.3

print("Model test:")
print(f"  z = {test_z}")
print(f"  H₀ = {test_H0} km/s/Mpc, Ωₘ = {test_Om}")
print(f"  d_L = {luminosity_distance(test_z, test_H0, test_Om)} Mpc")
print(f"  μ = {distance_modulus(test_z, test_H0, test_Om)}")
print("\n✓ Model implementation complete!")

---
## Part 3: Bayesian Inference Setup

Now we define the components of Bayes' theorem.

### 1. Prior Distribution P(θ)

We'll use **flat (uniform) priors** over physically reasonable ranges:

- $H_0$: 50 to 90 km/s/Mpc
  - Lower than 50 is inconsistent with local measurements
  - Higher than 90 is inconsistent with CMB and structure formation

- $\Omega_m$: 0.1 to 0.5
  - Lower than 0.1 has too little matter (conflicts with structure formation)
  - Higher than 0.5 conflicts with CMB, would give $\Omega_\Lambda < 0.5$

These are *weakly informative* priors - broad enough to not bias results, but exclude clearly unphysical values.

### 2. Likelihood Function P(D|θ)

For a single supernova with measurements $(z_i^{\text{obs}}, \mu_i^{\text{obs}})$ and uncertainties $(\sigma_{z,i}, \sigma_{\mu,i})$:

**Method**: We marginalize over the true redshift $z_i^{\text{true}}$:

$$P(\mu_i^{\text{obs}}, z_i^{\text{obs}} | \theta) = \int P(\mu_i^{\text{obs}} | z_i^{\text{true}}, \theta) \cdot P(z_i^{\text{obs}} | z_i^{\text{true}}) \cdot P(z_i^{\text{true}}) dz_i^{\text{true}}$$

**Assumptions**:
1. Gaussian errors: $P(z_i^{\text{obs}} | z_i^{\text{true}}) = \mathcal{N}(z_i^{\text{true}}, \sigma_{z,i}^2)$
2. Gaussian errors: $P(\mu_i^{\text{obs}} | z_i^{\text{true}}, \theta) = \mathcal{N}(\mu_{\text{model}}(z_i^{\text{true}}, \theta), \sigma_{\mu,i}^2)$
3. Flat prior on $z_i^{\text{true}}$ (uninformative)

**In practice**: We'll numerically integrate over possible true redshifts near each observed redshift.

**Total likelihood**: Product over all supernovae (assuming independence):

$$\mathcal{L}(\theta) = \prod_{i=1}^{N} P(\mu_i^{\text{obs}}, z_i^{\text{obs}} | \theta)$$

We work with **log-likelihood** for numerical stability:

$$\ln \mathcal{L}(\theta) = \sum_{i=1}^{N} \ln P(\mu_i^{\text{obs}}, z_i^{\text{obs}} | \theta)$$

In [ ]:
def log_prior(theta):
    """
    Log prior probability.

    Flat priors over reasonable ranges:
    - H0: [50, 90] km/s/Mpc
    - Omega_m: [0.1, 0.5]

    Parameters:
    -----------
    theta : array-like
        [H0, Omega_m]

    Returns:
    --------
    log_prior : float
        Log prior probability (0 if inside prior range, -inf if outside)
    """
    H0, Omega_m = theta

    # Check if parameters are in valid range
    if 50 < H0 < 90 and 0.1 < Omega_m < 0.5:
        return 0.0  # Flat prior (log(1) = 0)
    else:
        return -np.inf  # Outside prior range

def log_likelihood_single(z_obs_i, mu_obs_i, sigma_z_i, sigma_mu_i, H0, Omega_m):
    """
    Log likelihood for a single supernova, marginalizing over true redshift.

    We integrate over possible true redshifts z_true near z_obs.

    Parameters:
    -----------
    z_obs_i : float
        Observed redshift
    mu_obs_i : float
        Observed distance modulus
    sigma_z_i : float
        Redshift uncertainty
    sigma_mu_i : float
        Distance modulus uncertainty
    H0 : float
        Hubble constant
    Omega_m : float
        Matter density parameter

    Returns:
    --------
    log_L : float
        Log likelihood for this supernova
    """
    # Integration range: ±5σ around observed redshift
    z_min = max(0.001, z_obs_i - 5 * sigma_z_i)  # Don't go below z=0
    z_max = z_obs_i + 5 * sigma_z_i

    # Grid for numerical integration
    n_points = 20  # Trade-off between accuracy and speed
    z_true_grid = np.linspace(z_min, z_max, n_points)
    dz = z_true_grid[1] - z_true_grid[0]

    # Compute integrand at each grid point
    integrand = np.zeros(n_points)
    for j, z_true in enumerate(z_true_grid):
        # Model prediction at true redshift
        mu_model = distance_modulus(z_true, H0, Omega_m)

        # P(mu_obs | z_true, theta) - Gaussian
        log_p_mu = -0.5 * ((mu_obs_i - mu_model) / sigma_mu_i)**2

        # P(z_obs | z_true) - Gaussian
        log_p_z = -0.5 * ((z_obs_i - z_true) / sigma_z_i)**2

        # Combined probability (in log space, add; in linear space, multiply)
        integrand[j] = np.exp(log_p_mu + log_p_z)

    # Numerical integration (trapezoidal rule)
    integral = np.trapezoid(integrand, dx=dz)

    # Return log of integral (with numerical stability check)
    if integral > 0:
        return np.log(integral)
    else:
        return -np.inf

def log_likelihood(theta, z_obs, mu_obs, sigma_z, sigma_mu):
    H0, Omega_m = theta
    # Sum log likelihoods for all supernovae
    log_L = 0.0
    for i in range(len(z_obs)):
        log_L += log_likelihood_single(
            z_obs[i], mu_obs[i], sigma_z[i], sigma_mu[i], H0, Omega_m
        )
    return log_L

def log_probability(theta, z_obs, mu_obs, sigma_z, sigma_mu):
    """
    Log posterior probability (up to normalization constant).

    log P(theta | data) = log P(data | theta) + log P(theta) + const

    Parameters:
    -----------
    theta : array-like
        [H0, Omega_m]
    z_obs, mu_obs, sigma_z, sigma_mu : arrays
        Data and uncertainties

    Returns:
    --------
    log_prob : float
        Log posterior probability
    """
    # Check prior first (fast rejection of bad parameters)
    lp = log_prior(theta)
    if not np.isfinite(lp):
        return -np.inf

    # Add log likelihood
    ll = log_likelihood(theta, z_obs, mu_obs, sigma_z, sigma_mu)

    return lp + ll

# Test the functions
test_theta = [70, 0.3]
print("Testing Bayesian functions:")
print(f"  θ = {test_theta}")
print(f"  log_prior(θ) = {log_prior(test_theta)}")
print(f"  log_likelihood (first SN) = {log_likelihood_single(z_obs[0], mu_obs[0], sigma_z[0], sigma_mu[0], *test_theta):.3f}")
print("\n✓ Bayesian functions ready!")

---
## Part 4: Running MCMC with emcee

Now we'll use emcee to sample from our posterior distribution. MCMC takes a while because each likelihood evaluation requires a numerical integration, and there are many data points and steps needed to explore the parameter range.

### MCMC Parameters

- **n_walkers**: Number of parallel walkers (chains)
  - Rule of thumb: ≥ 2 × n_parameters
  - We'll use 32 walkers for 2 parameters

- **n_steps**: Number of steps each walker takes
  - Need enough to converge and sample posterior well
  - We'll use 2000 steps (then check convergence)

- **Initial positions**: Where walkers start
  - Should be near expected values but with some spread
  - We'll start near H₀=70, Ωₘ=0.3 with small perturbations

In fact, **this takes too long** for 1700 supernovae. I estimated it as about 20 hours. Instead try running with just 10 supernovae. The supernovae are sorted in terms of redshift, so choose only the last 10 data points, as those are most constraining, and let's see what we get.

In [ ]:
# For faster testing, use a subset of data
# write code here to select only last 10 data points for:
#    z_obs, mu_obs, sigma_z, sigma_mu





In [ ]:
print(f"Running MCMC with {len(z_obs)} supernovae...")
print("This will take a few minutes... or hours... or days ¯\_(ツ)_/¯. \n")

# MCMC setup
n_dim = 2  # Number of parameters (H0, Omega_m)
n_walkers = 16  # Number of walkers (should be >> n_dim)
n_steps = 1000  # Number of steps per walker

# Initial positions for walkers
# Start near expected values with small random perturbations
initial_H0 = 70
initial_Om = 0.3
pos_spread = 1e-4  # Small spread

initial_pos = np.array([initial_H0, initial_Om]) + pos_spread * np.random.randn(n_walkers, n_dim)

# Ensure all initial positions are within prior range
for i in range(n_walkers):
    while log_prior(initial_pos[i]) == -np.inf:
        initial_pos[i] = np.array([initial_H0, initial_Om]) + pos_spread * np.random.randn(n_dim)

print(f"MCMC Configuration:")
print(f"  Parameters: H₀, Ωₘ")
print(f"  Walkers: {n_walkers}")
print(f"  Steps: {n_steps}")
print(f"  Total samples: {n_walkers * n_steps}")
print(f"  Initial position: H₀≈{initial_H0}, Ωₘ≈{initial_Om}")

# Initialize sampler
sampler = emcee.EnsembleSampler(
    n_walkers, n_dim, log_probability,
    args=(z_obs, mu_obs, sigma_z, sigma_mu)
)

# Run MCMC
print(f"\nRunning MCMC...")
print("(This may take 5-10 minutes depending on data size)")
sampler.run_mcmc(initial_pos, n_steps, progress=True)

print("\n✓ MCMC sampling complete!")

---
## Part 5: Analyzing the Results

Now let's examine our MCMC chains and extract parameter estimates.

In [ ]:
# Get the chain
# Shape: (n_walkers, n_steps, n_dim)
samples = sampler.get_chain()

print(f"Chain shape: {samples.shape}")
print(f"  {samples.shape[0]} walkers")
print(f"  {samples.shape[1]} steps")
print(f"  {samples.shape[2]} parameters")

# Plot the chains to check convergence
fig, axes = plt.subplots(2, figsize=(10, 7), sharex=True)

labels = ["H₀ [km/s/Mpc]", "Ωₘ"]
for i in range(n_dim):
    ax = axes[i]
    ax.plot(samples[:, :, i], "k", alpha=0.3)
    ax.set_ylabel(labels[i], fontsize=14)
    ax.yaxis.set_label_coords(-0.1, 0.5)

axes[-1].set_xlabel("Step Number", fontsize=14)
fig.suptitle("MCMC Chain Traces", fontsize=16)
plt.tight_layout()
plt.show()

print("\n📊 Trace Plots:")
print("  - Each thin line is one walker")
print("  - Should see 'hairy caterpillar' pattern")
print("  - Early steps are 'burn-in' (exploring from initial position)")
print("  - Later steps sample the posterior")

### Burn-in and Thinning

Looking at the traces, the chains are not long enough (only 1,000 steps) to be able to tell when the "burn-in" is done from the autocorrelation. Instead inspect by eye and set the point at which it looks like the MCMC converged. Discard any early steps where the chains are clearly still moving to the center of the posterior distribution from the initial positions. Our final chains should just look like random walks.

In [ ]:
burnin = #
thin = 15

# Get flattened chain (all walkers combined, after burn-in and thinning)
flat_samples = sampler.get_chain(discard=burnin, thin=thin, flat=True)

print(f"\nFinal sample:")
print(f"  Shape: {flat_samples.shape}")
print(f"  Total samples: {flat_samples.shape[0]}")
print(f"  (After discarding {burnin} burn-in steps and thinning by {thin})")

---
## Part 6: Parameter Estimates

Now we can extract parameter estimates from our posterior samples.

In [ ]:
# Extract parameters
H0_samples = flat_samples[:, 0]
Om_samples = flat_samples[:, 1]

# Calculate statistics
H0_median = np.median(H0_samples)
H0_std = np.std(H0_samples)
H0_16, H0_84 = np.percentile(H0_samples, [16, 84])

Om_median = np.median(Om_samples)
Om_std = np.std(Om_samples)
Om_16, Om_84 = np.percentile(Om_samples, [16, 84])

# Derived quantities
OL_median = 1 - Om_median
q0_median = 0.5 * Om_median - OL_median

print("="*70)
print("BAYESIAN PARAMETER ESTIMATES")
print("="*70)
print(f"\nH₀ = {H0_median:.2f} +{H0_84-H0_median:.2f} -{H0_median-H0_16:.2f} km/s/Mpc")
print(f"    (median ± std: {H0_median:.2f} ± {H0_std:.2f})")

print(f"\nΩₘ = {Om_median:.3f} +{Om_84-Om_median:.3f} -{Om_median-Om_16:.3f}")
print(f"    (median ± std: {Om_median:.3f} ± {Om_std:.3f})")

print(f"\nDerived quantities:")
print(f"  Ω_Λ = {OL_median:.3f} (= 1 - Ωₘ)")
print(f"  q₀ = {q0_median:.3f} (= Ωₘ/2 - Ω_Λ)")

print("="*70)

# Compare with literature
print(f"\nLiterature values:")
print(f"  H₀ ≈ 67-74 km/s/Mpc (tension between measurements!)")
print(f"  Ωₘ ≈ 0.31 (Planck 2018)")
print(f"  Ω_Λ ≈ 0.69")
print(f"  q₀ ≈ -0.54")

if data_source == "synthetic":
    print(f"\nTrue values (synthetic data):")
    print(f"  H₀ = 70 km/s/Mpc")
    print(f"  Ωₘ = 0.30")
    print(f"\n  Did we recover the true parameters? ✓" if abs(H0_median - 70) < 2 else "  Check convergence!")

### Corner Plot

A **corner plot** shows the full posterior distribution:
- Diagonal: 1D marginalized posteriors (histograms)
- Off-diagonal: 2D joint posteriors (contours)
- Shows correlations between parameters

In [ ]:
# Create corner plot
fig = corner.corner(
    flat_samples,
    labels=["$H_0$ [km/s/Mpc]", "$\Omega_m$"],
    truths=[70, 0.3] if data_source == "synthetic" else None,
    quantiles=[0.16, 0.5, 0.84],
    show_titles=True,
    title_kwargs={"fontsize": 12},
    label_kwargs={"fontsize": 14}
)

if data_source == "synthetic":
    fig.suptitle("Posterior Distribution (red lines = true values)", fontsize=16, y=1.02)
else:
    fig.suptitle("Posterior Distribution", fontsize=16, y=1.02)

plt.show()

print("\n📊 Corner Plot Interpretation:")
print("  - Diagonal panels: 1D posterior for each parameter")
print("  - Off-diagonal: 2D joint posterior (contours show 1σ, 2σ levels)")
print("  - Are H₀ and Ωₘ correlated or independent?")
print("  - Are posteriors roughly Gaussian or skewed?")

---
## Part 7: Model Comparison

Let's visualize how well our model fits the data.

In [ ]:
# Plot data and model with posterior uncertainty

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Left: Full fit
ax1 = axes[0]

# Data
ax1.errorbar(z_obs, mu_obs, yerr=sigma_mu, xerr=sigma_z,
             fmt='o', alpha=0.4, markersize=3, elinewidth=0.5,
             color='gray', label='Data')

# Model curves from posterior samples
z_model = np.linspace(0, max(z_obs), 300)

# Plot several random samples from posterior (to show uncertainty)
n_plot = 100
indices = np.random.randint(len(flat_samples), size=n_plot)

for idx in indices:
    H0_sample, Om_sample = flat_samples[idx]
    mu_model = distance_modulus(z_model, H0_sample, Om_sample)
    ax1.plot(z_model, mu_model, 'b-', alpha=0.02, linewidth=0.5)

# Best fit (median)
mu_best = distance_modulus(z_model, H0_median, Om_median)
ax1.plot(z_model, mu_best, 'r-', linewidth=2.5,
         label=f'Best fit: H₀={H0_median:.1f}, Ωₘ={Om_median:.2f}')

ax1.set_xlabel('Redshift (z)', fontsize=14)
ax1.set_ylabel('Distance Modulus (μ)', fontsize=14)
ax1.set_title('Model Fit to Data', fontsize=16)
ax1.legend(fontsize=12)
ax1.grid(True, alpha=0.3)

# Right: Residuals
ax2 = axes[1]

mu_pred = distance_modulus(z_obs, H0_median, Om_median)
residuals = mu_obs - mu_pred

ax2.errorbar(z_obs, residuals, yerr=sigma_mu,
             fmt='o', alpha=0.5, markersize=3, elinewidth=0.5)
ax2.axhline(0, color='red', linestyle='--', linewidth=2)
ax2.set_xlabel('Redshift (z)', fontsize=14)
ax2.set_ylabel('Residuals (μ_obs - μ_model)', fontsize=14)
ax2.set_title('Fit Residuals', fontsize=16)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Repeat with 200 data points

We can see that 10 data points does not cut it! Repeat this exercise again, but instead use the '''sampler''' data produced on 200 data points. You can download this from our class github using the code below.

In [121]:
!wget -p https://raw.githubusercontent.com/profhewitt/ast3414/refs/heads/main/emcee_200samples.pkl

--2026-02-16 20:38:37--  https://raw.githubusercontent.com/profhewitt/ast3414/refs/heads/main/emcee_200samples.pkl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 512141 (500K) [application/octet-stream]
Saving to: ‘raw.githubusercontent.com/profhewitt/ast3414/refs/heads/main/emcee_200samples.pkl’

raw.githubuserconte 100%[===================>] 500.14K  --.-KB/s    in 0.04s   

2026-02-16 20:38:38 (13.3 MB/s) - ‘raw.githubusercontent.com/profhewitt/ast3414/refs/heads/main/emcee_200samples.pkl’ saved [512141/512141]

FINISHED --2026-02-16 20:38:38--
Total wall clock time: 0.1s
Downloaded: 1 files, 500K in 0.04s (13.3 MB/s)


In [122]:
# Code to load in Hewitt's MCMC run
import pickle
# Open file in binary write mode ('wb') and dump the object
with open('emcee_200samples.pkl', 'rb') as outfile:
    samples = pickle.load(outfile)

In [123]:
ADD YOUR CODE HERE

(2000, 16, 2)

---
# Discussion Questions

1. How senstive are the results to the number of data points? How much do you think your results would improve using all 1700 supernovae? Justify your speculation.

2. How would you need to modify the code to fit a non-flat universe (include either curvature or $\Omega_\Lambda$ as a free parameter)? You do not have to successfully re-run your model with this modification, but describe below how it could be done, or edit a bit of code.